In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'food-101:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F8544%2F11959%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240726%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240726T162454Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1318d32adeaceff724e51a45cd8ef5d3836b2b5577d62f064d99469199c136c9a9974e490b5c5b842d2734739c2ae5b8df64fd9b6aef8529cd65ccc58bc912a1662ad298442ee6b59300c9da10055d87ce9032e478eea3ccc78ffe6e3214a0753e2c2ce3712b27611039faa3c3ec3d1172c2cbb565acea6f0099d36f5d9878314d335396a82aa36792448ed26c7d378f8e5d42f9eb3566c8fbc16d0e6442b83919870c3e112d6fef1c3bfe085fae1115e130ea865132c55de435edb3b7f4b9b3c3ae6a22c03b8b33ec23254316f7e008a41fddfe04ed1d9fec0fe79991e655fcad35561d0f9aedb1efb4d79d6fedc1711b56e3855d1fe986e4b5de3ad98069af'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import efficientnet
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.applications.xception import Xception

In [ ]:
%cd /kaggle/input/food-101/

In [ ]:
!head 'food-101/food-101/meta/train.txt'

In [ ]:
train_df = pd.read_csv('food-101/food-101/meta/train.txt', header = None, names=['path'])
test_df = pd.read_csv('food-101/food-101/meta/test.txt', header = None, names=['path'])
len(train_df)

In [ ]:
def spliter(data, class_or_id='id'):
    if class_or_id.upper() == 'CLASS':
        output = data.split('/')[0]

    else:
        output = data.split('/')[-1]
    return output

In [ ]:
train_df['label'] = train_df['path'].map(lambda x: spliter(data = x, class_or_id = 'Class'))
train_df['idx'] = train_df['path'].map(lambda x: spliter(x))
test_df['label'] = test_df['path'].map(lambda x: spliter(x, 'class'))
test_df['idx'] = test_df['path'].map(lambda x: spliter(x))

In [ ]:
food_25 = train_df['label'].unique()[:20]
print(food_25)

In [ ]:
list_ = []
for f in food_25:
    list_.append(f.upper())
food_21 = [food.upper() for food in food_25]

In [ ]:
def prepare_data(label):
    if label.upper() in food_25:
        return label
    else:
        return 'others'

In [ ]:
train_df['label'] = train_df['label'].map(lambda x: prepare_data(x))
test_df['label'] = test_df['label'].map(lambda x: prepare_data(x))
print(train_df['label'].unique())

In [ ]:
def Adding_Path(path):
    return 'food-101/food-101/images/'+path+'.jpg'
train_df['path'] = train_df[['path']].apply(Adding_Path, axis=1)
test_df['path'] = test_df[['path']].apply(Adding_Path, axis=1)

In [ ]:
test_images = plt.imread(test_df['path'].iloc[118])
test_images = test_images/200.
test_images.shape

In [ ]:
train_images = plt.imread(train_df['path'].iloc[105])
plt.imshow(train_images)
train_images.shape

In [ ]:
datagen = ImageDataGenerator(rescale=1. / 255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )
#test_gen = ImageDataGenerator(rescale=1. / 255)

test_gen = datagen.flow_from_dataframe(dataframe = test_df, directory=None, x_col='path', y_col='label',
    weight_col=None, target_size=(256, 256), color_mode='rgb',
    classes=None, class_mode='categorical', batch_size=32,shuffle=False)

train_gen = datagen.flow_from_dataframe(dataframe = train_df, directory=None, x_col='path', y_col='label',
    weight_col=None, target_size=(256, 256), color_mode='rgb',
    classes=None, class_mode='categorical', batch_size=32, shuffle=True)

In [ ]:
inception = Xception(weights='imagenet', include_top=False)
x = inception.output
x = GlobalAveragePooling2D()(x)
x = Dense(256,activation='relu')(x)
x = Dense(128,activation='relu')(x)
x = Dropout(0.2)(x)

predictions = Dense(21, activation='softmax')(x)

model = Model(inputs=inception.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_gen,
                    steps_per_epoch = len(train_gen) // 3,
                    epochs=2,
                    verbose=1)

In [ ]:
print(history.history.keys())

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['loss'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Training Accuracy', 'Training Loss'], loc='upper left')
plt.show()

In [ ]:
results = model.evaluate(test_gen, batch_size=3)
print(results)
#import sys
#sys.getsizeof(test_gen)